# 📊 Stock Market Analysis

This notebook performs complete EDA and analysis on the uploaded `stocks.csv` dataset.

Steps include:
- Data loading & cleaning
- Exploratory Data Analysis (EDA)
- Feature engineering (returns, moving averages, volatility)
- Correlation analysis
- Data visualization

---

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler

plt.rcParams['figure.dpi'] = 120
sns.set(style='whitegrid')


In [ ]:

# Load the dataset
data = pd.read_csv('/mnt/data/stocks.csv')
print('Shape:', data.shape)
data.head()


In [ ]:

# Data Cleaning
data.columns = [c.strip() for c in data.columns]
data['Date'] = pd.to_datetime(data['Date'])
data = data.dropna()
data = data.sort_values(['Ticker', 'Date']).reset_index(drop=True)
data.info()


In [ ]:

# Exploratory Data Analysis
print(data.describe())
sns.pairplot(data, hue='Ticker')
plt.show()

sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()


In [ ]:

# Feature Engineering - Moving Averages and Volatility
data['Return'] = data.groupby('Ticker')['Close'].pct_change()
data['LogReturn'] = np.log(data['Close'] / data['Close'].shift(1))
for w in [7, 21, 50]:
    data[f'MA_{w}'] = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(w).mean())
    data[f'Vol_{w}'] = data.groupby('Ticker')['LogReturn'].transform(lambda x: x.rolling(w).std()) * np.sqrt(252)

data.head()


In [ ]:

# Visualization of Price Trends
plt.figure(figsize=(10,5))
for t, g in data.groupby('Ticker'):
    plt.plot(g['Date'], g['Close'], label=t)
plt.legend()
plt.title('Closing Price Over Time')
plt.show()

# Histogram of Returns
sns.histplot(data['Return'].dropna(), bins=50, kde=True)
plt.title('Return Distribution')
plt.show()


In [ ]:

# Correlation Analysis
pivot_df = data.pivot(index='Date', columns='Ticker', values='Close')
corr = pivot_df.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Price Correlation Between Companies')
plt.show()
